# DETAILED KNOWLEDGE OF THE
## FEATURE EXTRACTION AND DATASET CREATION

Firstly we have collected the audios from VoxCeleb and also a csv file(vox1_meta.csv).

The mentioned csv file will contain the gender of the persons speaking with the audio id's.

The gender and the id's are separated from this csv file and store in another csv file.

The Dataset is created in such a way that the most of the male and female voice are taken into account.

The features are extracted from audio in the function get_features.

Then the dataset is created with 16 columns.

The last column 'LABEL'is added in the dataset with the help of the dataset created from vox1_meta.csv.

Finally the Dataset is saved as dataset.csv.

While checking the gender of the person speaking in the audio,we have noticed that in audios where

a female with a deep voice result in wrong prediction.

So we have added some more audios to the existing dataset which we have collected from some of our 

friends. 

Click on the link to check how we have added more audios to the existing dataset.

<a href="http://localhost:8888/notebooks/Desktop/Final%20Year%20Project/DatasetCreation2.ipynb">CLICK HERE</a>

In [1]:
#IMPORT THE REQUIRED PACKAGES
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.io import wavfile 
from scipy.stats import stats
import multiprocessing
import concurrent.futures
import time
import librosa

In [2]:
#Read the csv file obtained from VoxCeleb to seaprate the genders 
read_file=pd.read_csv(r'C:\\Users\\SAGAR\\Desktop\\Final Year Project\\csv_dataset\\vox1_meta.csv')
read_file.head()

,VoxCeleb1 ID VGGFace1 ID Gender Nationality Set
0,id10001\tA.J._Buckley\tm\tIreland\tdev
1,id10002\tA.R._Rahman\tm\tIndia\tdev
2,id10003\tAamir_Khan\tm\tIndia\tdev
3,id10004\tAaron_Tveit\tm\tUSA\tdev
4,id10005\tAaron_Yoo\tm\tUSA\tdev


In [3]:
line=read_file.to_numpy()#convert the csv to numpy array
line

array([['id10001\tA.J._Buckley\tm\tIreland\tdev'],
       ['id10002\tA.R._Rahman\tm\tIndia\tdev'],
       ['id10003\tAamir_Khan\tm\tIndia\tdev'],
       ...,
       ['id11249\tZack_Snyder\tm\tUSA\tdev'],
       ['id11250\tZoe_Saldana\tf\tUSA\tdev'],
       ['id11251\tZulay_Henao\tf\tUSA\tdev']], dtype=object)

In [4]:
vgg_id=[]
vgg_name=[]
vgg_gender=[]
vgg_nation=[]
vgg_set=[]
for i in line:
    s=i[0]
    vg_id,vg_name,vg_gender,vg_nation,vg_set=s.split("\t")#split each array element on the basis of '/'
    #from this csv file we are only taking the audio names for further use
    #the gender and the nation
    vgg_id.append(vg_id)
    vgg_gender.append(vg_gender)
    vgg_nation.append(vg_nation)
csv_dict=({"VGG_ID":vgg_id,"LABEL":vgg_gender,"COUNTRY":vgg_nation})
csv_df=pd.DataFrame(csv_dict)#DataFrame is created with these three values  

In [5]:
array_label=csv_df.to_numpy()
id_1=[]
label_1=[]
country_1=[]
id_0=[]
label_0=[]
country_0=[]
for i in array_label:
    label=i[1]
    if label=='m':
        id_1.append(i[0])
        label_1.append(i[1])
        country_1.append(i[2])
    elif label=='f':
        id_0.append(i[0])
        label_0.append(i[1])
        country_0.append(i[2])
csv_label_1=({"VGG_ID":id_1,"LABEL":label_1,"COUNTRY":country_1}) 
csv_label_0=({"VGG_ID":id_0,"LABEL":label_0,"COUNTRY":country_0})
csv_label_1=pd.DataFrame(csv_label_1)
csv_label_0=pd.DataFrame(csv_label_0)

In [6]:
csv_label=pd.concat([csv_label_1,csv_label_0])
csv_label.index=range(len(csv_label))
csv_label.to_csv("C:\\Users\\SAGAR\\Desktop\\Final Year Project\\csv_dataset\\voice_dataset.csv",index=False)

In [7]:
def get_frequency(audio_file):
    #print(audio_file)
    sp_centroid=[]
    sp_flatness1=[]
    sp_rolloff1=[]
    sp_bandwidth1=[]
    freqArray1=[]
    freqArray2=[]
    pitch1=[]
    rate,data=wavfile.read(audio_file) 
    step=int(rate/5)
    #p=find_pitch(audio_file)
    
    x, sr = librosa.load(audio_file)
    spectral_centroids=librosa.feature.spectral_centroid(x,sr=sr)[0] #to find the spectral centroid of the audio
    flatness=librosa.feature.spectral_flatness(x)#to find the spectral flatness of the audio
    rolloff = librosa.feature.spectral_rolloff(x,sr=sr)#to find the spectral roll off of the audio
    bandwidth=librosa.feature.spectral_bandwidth(x,sr=sr)#to find the spectral bandwidth of the audio
    sp_centroid.append(np.mean(spectral_centroids))
    sp_flatness1.append(np.mean(flatness))
    sp_rolloff1.append(np.mean(rolloff))
    sp_bandwidth1.append(np.mean(bandwidth))
    #Find out the Dominating Frequency Using Fourier
    for j in range(0,len(data),step):
            fourier=np.fft.fft(data[j:j+step])
            freqs=np.fft.fftfreq(len(fourier))#freqs tells you the frequencies associated with the coefficients
            imax=np.argmax(np.abs(fourier))
            freq=freqs[imax]
            freq_in_hz=abs(freq *rate)
            freqArray1.append(freq_in_hz)
            x=[f for f in freqArray1 if 20<f<300 and not 46<f<66]
    freqArray2.append(x)
    sp_centroids=np.mean(sp_centroid)  
    sp_flatness=np.mean(sp_flatness1)
    sp_rolloff=np.mean(sp_rolloff1)
    sp_bandwidth=np.mean(sp_bandwidth1)
    freqArray=[item for sublist in freqArray2 for item in sublist]
    return freqArray,sp_centroids,sp_flatness,sp_rolloff,sp_bandwidth

In [8]:
def get_features(path):
    nobs1=[]
    index=[]
    mean1=[]
    variance1=[]
    skew1=[]
    kurtosis1=[]
    median1=[]
    mode1=[]
    std1=[]
    low1=[]
    peak1=[]
    q751=[]
    q251=[]
    iqr1=[]
    pitch1=[]
    low_pitch1=[]
    high_pitch1=[]
    sp_cent_1=[]
    sfm_1=[]
    sp_rolloff_1=[]
    sp_bandwidth_1=[]
    files=os.listdir(path)
    n=len(files)
    for i in range(0,n):
        path1=os.path.join(path,files[i])
        #print(path1)
        sound_file=os.listdir(path1)
        n1=len(sound_file)
        for j in range(n1):
            path2=os.path.join(path1,sound_file[j])
            audio_file=os.listdir(path2)
            n2=len(audio_file)
            freq=[]
            sp_cent1=[]
            sfm1=[]
            sp_rolloff1=[]
            sp_band1=[]
            path3=[]
            pitch_1=[]
            low_pitch1=[]
            high_pitch1=[]
            print(path2)
            for r in audio_file:
                p=os.path.join(path2,r)
                path3.append(p)
           #MultiProcessing is done to reduce the time it takes to extract features from so many audios     
            with concurrent.futures.ThreadPoolExecutor() as exec:
                for freq1,sp,sfm2,sroll,sband in exec.map(get_frequency,path3):
                    freq.append(freq1)
                    sp_cent1.append(sp)
                    sfm1.append(sfm2)
                    sp_rolloff1.append(sroll)
                    sp_band1.append(sband)
            freqArray=[item for sublist in freq for item in sublist]
            sp_cent=np.mean(sp_cent1)#to find the mean of all the spectral centroids obtained of an audio
            sfm=np.mean(sfm1)#to find the mean of all the spectral flatness obtained of an audio
            sp_rolloff=np.mean(sp_rolloff1)#to find the mean of all the spectral roll off obtained of an audio
            sp_bandwidth=np.mean(sp_band1)#to find the mean of all the spectral bandwidth obtained of an audio
            nobs,minmax,mean,variance,skew,kurtosis=stats.describe(freqArray)#to find out the dominating frquency,skewness and kurtosis of the audio
            mode=stats.mode(freqArray).mode[0]#to find out the mode of the frequencies of the audio
            std=np.std(freqArray)#to find out the standard deviation 
            low,peak=minmax#to find out the min and max dominating frequency
            q75,q25=np.percentile(freqArray, [75 ,25])#to find out first quartile and second quartile
            iqr=q75-q25#to find out the interquartile range
            median=np.median(freqArray)#to find out the median
        
            #Here the array for each column in the dataset is created by appending the value obtained for above
            #for the given audio
            index.append(files[i])
            mean1.append(mean/1000)
            skew1.append(skew)
            kurtosis1.append(kurtosis)
            median1.append(median/1000)
            mode1.append(mode/1000)
            std1.append(std/1000)
            low1.append(low/1000)
            peak1.append(peak/1000)
            q751.append(q75/1000)
            q251.append(q25/1000)
            iqr1.append(iqr/1000)
            sp_cent_1.append(sp_cent)
            sfm_1.append(sfm)
            sp_rolloff_1.append(sp_rolloff)
            sp_bandwidth_1.append(sp_bandwidth)
    d=({'ID':index,'meanfreq':mean1,'sd':std1,'median':median1,'Q25':q251,'Q75':q751,'iqr':iqr1,'skew':skew1,'kurt':kurtosis1,'mode':mode1,'low':low1,'peak':peak1,'centroid':sp_cent_1,'flatness':sfm_1,'spectral rolloff':sp_rolloff_1,'spectral bandwidth':sp_bandwidth_1})
    df=pd.DataFrame(data=d)
    return df    

In [ ]:
path="D:\\training_dataset"
data=get_features(path)
files=data.to_numpy()
print("   NUMBER OF AUDIO FILES IN DATASET:")
print("=======================================")
n=len(files)
print(n)

D:\training_dataset\id10001\1zcIwhmdeo4
D:\training_dataset\id10001\7gWzIy6yIIk
D:\training_dataset\id10001\7w0IBEWc9Qw
D:\training_dataset\id10001\9mQ11vBs1wc
D:\training_dataset\id10001\DtdEYdViWdw
D:\training_dataset\id10001\eWIX7sfn-M0
D:\training_dataset\id10001\J9lHsKG98U8
D:\training_dataset\id10001\utrA-v8pPm4
D:\training_dataset\id10001\Y8hIVOBuels
D:\training_dataset\id10001\zELwAz2W6hM
D:\training_dataset\id10002\0_laIeN-Q44
D:\training_dataset\id10002\6WO410QOeuo
D:\training_dataset\id10002\C7k7C-PDvAA
D:\training_dataset\id10002\cMGEuZ1zqXk
D:\training_dataset\id10002\eNc4LrrvV80
D:\training_dataset\id10002\gaQqIoV_aLY
D:\training_dataset\id10002\Mpr9wqUuLQA
D:\training_dataset\id10002\QanuGhOhb9A
D:\training_dataset\id10002\QnnjJ9i5WFs
D:\training_dataset\id10002\RLKKsYiCMvc
D:\training_dataset\id10002\SI4D2_YXvBE
D:\training_dataset\id10002\TqUbiOgEb0w
D:\training_dataset\id10002\VMaXdHLz5Bk
D:\training_dataset\id10002\w9H-ZMvdE9M
D:\training_dataset\id10002\wd_7oYV4dsU


C:\Users\SAGAR\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\SAGAR\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


D:\training_dataset\id10002\xTV-jFAUKcw
D:\training_dataset\id10002\Y2Gr1I2DO7M
D:\training_dataset\id10003\5ablueV_1tw
D:\training_dataset\id10003\A7Hh1WKmHsg
D:\training_dataset\id10003\A8SBCxYzJgs
D:\training_dataset\id10003\bDxy7bnj_bc
D:\training_dataset\id10003\BQxxhq4539A
D:\training_dataset\id10003\EGPV-Xa0LGk
D:\training_dataset\id10003\E_6MjfYr0sQ
D:\training_dataset\id10003\FfmnkloV_zg
D:\training_dataset\id10003\K5zRxtXc27s
D:\training_dataset\id10003\na8-QEFmj44
D:\training_dataset\id10003\NC70RWJDnMg
D:\training_dataset\id10003\tCq2LcKO6xY
D:\training_dataset\id10003\Tzn91xwBaWE
D:\training_dataset\id10003\yzIXg93UOIM
D:\training_dataset\id10003\_JpHD6VnJ3I
D:\training_dataset\id10006\0otHlFztX8I
D:\training_dataset\id10006\3MwyuwaVE50
D:\training_dataset\id10006\3RybHF5mX78
D:\training_dataset\id10006\5PBimrKDrJQ
D:\training_dataset\id10006\5tGaUGO_z50
D:\training_dataset\id10006\7qUfkhbDaqc
D:\training_dataset\id10006\7W9goO0lNrA
D:\training_dataset\id10006\88biLNZtnag


D:\training_dataset\id10026\dQhROKhai8Y
D:\training_dataset\id10026\DXN_rEFztlc
D:\training_dataset\id10026\JzrVApWpAdo
D:\training_dataset\id10026\kpkaaj2jx6k
D:\training_dataset\id10026\KzCToed2Rcc
D:\training_dataset\id10026\TuftZsOgdzs
D:\training_dataset\id10026\vkHXIe40wns
D:\training_dataset\id10026\Vs-FvXU_GBE
D:\training_dataset\id10026\WBjWoYv8WcI
D:\training_dataset\id10026\yczc53tYhS8
D:\training_dataset\id10027\50IAfJCypFI
D:\training_dataset\id10027\69_QrKoy-GA
D:\training_dataset\id10027\6cXKzqLQYfQ
D:\training_dataset\id10027\ccbXWh4og34
D:\training_dataset\id10027\E9_VY1Y5uL0
D:\training_dataset\id10027\FATQ1UkoIH4
D:\training_dataset\id10027\g2m1E3C9wY4
D:\training_dataset\id10027\mnJX0ydvfBw
D:\training_dataset\id10027\N4vE1gwQQQA
D:\training_dataset\id10027\orwnPXD5JIU
D:\training_dataset\id10027\P-7wcXaLUUs
D:\training_dataset\id10027\qWdjhdcaHyk
D:\training_dataset\id10027\QZuA-h2TORw
D:\training_dataset\id10027\UFrqp10LwLs
D:\training_dataset\id10027\USjCbjIpTuI


D:\training_dataset\id10041\3t-RnqGXDrM
D:\training_dataset\id10041\5g4NwozVJpA
D:\training_dataset\id10041\6TmD9opLBY8
D:\training_dataset\id10041\9fJl5wUSbMM
D:\training_dataset\id10041\9THGEtI-rv8
D:\training_dataset\id10041\airbUybc41Y
D:\training_dataset\id10041\Ap16js05dA8
D:\training_dataset\id10041\bkT-M07CrjA
D:\training_dataset\id10041\D3dHTB6l9r0
D:\training_dataset\id10041\eUXTdQY-XYE
D:\training_dataset\id10041\g_v7-sRZGWs
D:\training_dataset\id10041\HBF8LIr8RG4
D:\training_dataset\id10041\hJjhdy2R-xg
D:\training_dataset\id10041\jar0wj_nfJk
D:\training_dataset\id10041\LNlufCgIx_E
D:\training_dataset\id10041\LrXEcxBXPWA
D:\training_dataset\id10041\LvoztHyRXy8
D:\training_dataset\id10041\mcpkLfUBzeU
D:\training_dataset\id10041\n9rD9-kU2Mw
D:\training_dataset\id10041\nUH8iI0t4rE
D:\training_dataset\id10041\oKMQz0cxdQw
D:\training_dataset\id10041\Pg20jX3taeA
D:\training_dataset\id10041\PZni1fi5MTQ
D:\training_dataset\id10041\qsX7s9M3lbk
D:\training_dataset\id10041\rzxZntDzey0


...

D:\training_dataset\id10297\o1EKUvjvNHY
D:\training_dataset\id10297\p08spQszEYU
D:\training_dataset\id10297\utAY0zpsv1U
D:\training_dataset\id10297\uxHwuJ59xNk
D:\training_dataset\id10298\3rge7xFekcQ
D:\training_dataset\id10298\6qFnVechX9o
D:\training_dataset\id10298\8v0-r8HKI_0
D:\training_dataset\id10298\9o3HnyKpHLM
D:\training_dataset\id10298\a0S-epXYJME
D:\training_dataset\id10298\aEuSKFj0xo8
D:\training_dataset\id10298\Cjt5DP6qc5Q
D:\training_dataset\id10298\DWT9P35cXT4
D:\training_dataset\id10298\f5eaTNf7-io
D:\training_dataset\id10298\gNR5NHzd3o0
D:\training_dataset\id10298\H5X-KaeTzEM
D:\training_dataset\id10298\hjvQiiG71rM
D:\training_dataset\id10298\hSC1Yb953Sg
D:\training_dataset\id10298\KaNukQi4imU
D:\training_dataset\id10298\KnOFOXpwmp0
D:\training_dataset\id10298\qrJ0fQJP5gs
D:\training_dataset\id10298\SCYPNx1H2pM
D:\training_dataset\id10298\U_1vfIVOigA
D:\training_dataset\id10298\VHtcwtdPjpE
D:\training_dataset\id10299\752GDDvGPCA
D:\training_dataset\id10299\DdZRjmvZKbI


D:\training_dataset\id10308\sy19cFuDvX8
D:\training_dataset\id10308\Tzs_CTbHT9Y
D:\training_dataset\id10308\vbxURBaCGr0
D:\training_dataset\id10308\We7oIBrKQKg
D:\training_dataset\id10308\XQWpVt5n3Ic
D:\training_dataset\id10308\ygGJs_Rh7ds
D:\training_dataset\id10309\0b1inHMAr6o
D:\training_dataset\id10309\0cYFdtyWVds
D:\training_dataset\id10309\0wLebLY6tcM
D:\training_dataset\id10309\3qX7bbJ17lY
D:\training_dataset\id10309\e-IdJ8a4gy4
D:\training_dataset\id10309\GyJHpVQmcvc
D:\training_dataset\id10309\MJ3skMKNBGM
D:\training_dataset\id10309\nMZkxUQ1RqI
D:\training_dataset\id10309\pwfqGqgezH4
D:\training_dataset\id10309\qFrRfhWombs
D:\training_dataset\id10309\RkN41E3gRpc
D:\training_dataset\id10309\rqaAm4bEsXc
D:\training_dataset\id10309\RWpuDFuKFjY
D:\training_dataset\id10309\rxnN8thYzEQ
D:\training_dataset\id10309\tGEWD2GaiDw
D:\training_dataset\id10309\uTOEzdTJNmU
D:\training_dataset\id10309\vobW27_-JyQ
D:\training_dataset\id10309\VxiCsd0gA6o
D:\training_dataset\id10309\V_tzoDBiP8A


D:\training_dataset\id10321\FWl8i5z7Omc
D:\training_dataset\id10321\GtUep_eTQ7U
D:\training_dataset\id10321\hBjeJUlFwsk
D:\training_dataset\id10321\lHfkWwCIT1E
D:\training_dataset\id10321\ljoqep7Io4Q
D:\training_dataset\id10321\lVrq3CHePwo
D:\training_dataset\id10321\n8VLoG1dTWM
D:\training_dataset\id10321\o4sTVkDiA1I
D:\training_dataset\id10321\oinBZMkxgts
D:\training_dataset\id10321\qAkrKOCOAdc
D:\training_dataset\id10321\sM6t2znWSo0
D:\training_dataset\id10321\SpxA79ojPY4
D:\training_dataset\id10321\VCCUJbbbJ4A
D:\training_dataset\id10321\x6bYDSh6EQI
D:\training_dataset\id10321\_mUc_Xl3gdM
D:\training_dataset\id10323\137siDbXp0I
D:\training_dataset\id10323\6zlsqpaYJdI
D:\training_dataset\id10323\kmnrEbUqNpI
D:\training_dataset\id10323\nrShtAbues4
D:\training_dataset\id10323\oTbfU_4YKdM
D:\training_dataset\id10323\QNGljKaw7pI
D:\training_dataset\id10323\VG9nNnr_a2o
D:\training_dataset\id10323\yyquFD8EKbA
D:\training_dataset\id10323\_bWQM4g06NY
D:\training_dataset\id10324\1BN1Twr0pDM


D:\training_dataset\id10334\alA_YRxKZCg
D:\training_dataset\id10334\C_JlIlY7Pbg
D:\training_dataset\id10334\gdS156IsqgE
D:\training_dataset\id10334\S1EWztcS8OE
D:\training_dataset\id10334\SW59jYm9Inw
D:\training_dataset\id10334\sWyVJdigpjc
D:\training_dataset\id10334\VpmzPfEfmb8
D:\training_dataset\id10334\VQBnetylQTI
D:\training_dataset\id10334\x7wfgqZQT5M
D:\training_dataset\id10334\ylLusLP--Ak
D:\training_dataset\id10334\YrI8Bv-ukgI
D:\training_dataset\id10334\YVXdKnu0s6k
D:\training_dataset\id10334\ZhhqTUZeIlU
D:\training_dataset\id10335\3j5n_FdTUg4
D:\training_dataset\id10335\5OLzRQHT290
D:\training_dataset\id10335\6lFw-tpQPLA
D:\training_dataset\id10335\8vwmMOAb5F8
D:\training_dataset\id10335\AY9apGnb0eM
D:\training_dataset\id10335\dCdRPb2_lMg
D:\training_dataset\id10335\DofYixK7wUQ
D:\training_dataset\id10335\expqXRg1Uyk
D:\training_dataset\id10335\GmZQ6Y0FSPA
D:\training_dataset\id10335\I9Hq0ZJ65l0
D:\training_dataset\id10335\IeLbPRXujaU
D:\training_dataset\id10335\JGgisII8aP4


D:\training_dataset\id10348\htGoC9IiAPc
D:\training_dataset\id10348\J5FZ35Ja3hQ
D:\training_dataset\id10348\kDku13RPflI
D:\training_dataset\id10348\KIy0AGQSAJc
D:\training_dataset\id10348\L0y-9ignSgs
D:\training_dataset\id10348\MNKD9ZPx8OY
D:\training_dataset\id10348\UIkqT7O1Ij8
D:\training_dataset\id10348\ydO2ZO-6-yU
D:\training_dataset\id10349\8VlMBWsbL0o
D:\training_dataset\id10349\CkdpPS1E5Ks
D:\training_dataset\id10349\ebmuSH4EpLc
D:\training_dataset\id10349\gO_tKqtX4sc
D:\training_dataset\id10349\h2XXZOzcqzM
D:\training_dataset\id10349\jQglnsLZ3R0
D:\training_dataset\id10349\QGlexcvxGFo
D:\training_dataset\id10349\vwHG7YVqIpE
D:\training_dataset\id10350\5pa13peHVtk
D:\training_dataset\id10350\6o-LScd5k38
D:\training_dataset\id10350\BppiewHcORc
D:\training_dataset\id10350\Ok_1huCJP0o
D:\training_dataset\id10350\pYi_jIz_VSw
D:\training_dataset\id10350\ryGNVsMrgBs
D:\training_dataset\id10350\SHWdWC0GJYo
D:\training_dataset\id10350\sVAA-cQ6JaA
D:\training_dataset\id10350\sxmVLwz2dGU


In [ ]:
#The Dataset
data.head()

In [ ]:
#Now we have to label each row with the gender correponding to the 'ID'
csv_label=pd.read_csv("C:\\Users\\SAGAR\\Desktop\\Final Year Project\\csv_dataset\\voice_dataset.csv")
csv_file=csv_label.to_numpy()
csv_file

Firstly,we read the csv file which contain only the 'ID' and 'gender'.
Then the 'ID' of the csv file and the dataset is compared.
If they are equal,the gender 'M' is added to one dataframe and gender 'F' is added to another dataframe
Then both the dataframes are concatenated and then the entire dataset is saved as csv file.

In [ ]:
array_label=data.to_numpy()
id_1=[]
label_1=[]
pitch_1=[]
low_pitch_1=[]
high_pitch_1=[]
mean_1=[]
skew_1=[]
kurt_1=[]
median_1=[]
mode_1=[]
sd_1=[]
low_1=[]
peak_1=[]
q75_1=[]
q25_1=[]
iqr_1=[]
sp_cent_1=[]
sfm_1=[]
sp_rolloff_1=[]
sp_bandwidth_1=[]
id_0=[]
label_0=[]
pitch_0=[]
low_pitch_0=[]
high_pitch_0=[]
mean_0=[]
skew_0=[]
kurt_0=[]
median_0=[]
mode_0=[]
sd_0=[]
low_0=[]
peak_0=[]
q75_0=[]
q25_0=[]
iqr_0=[]
sp_cent_0=[]
sfm_0=[]
sp_rolloff_0=[]
sp_bandwidth_0=[]
for i in csv_file:
    label=i[1]
    id_i=i[0]
    for j in array_label:
        id_j=j[0]
        if id_i==id_j:
            if label=='m':
                id_1.append(id_j)
                mean_1.append(j[1])
                sd_1.append(j[2])
                median_1.append(j[3])
                q25_1.append(j[4])
                q75_1.append(j[5])
                iqr_1.append(j[6])
                skew_1.append(j[7])
                kurt_1.append(j[8])
                mode_1.append(j[9])
                low_1.append(j[10])
                peak_1.append(j[11])
                sp_cent_1.append(j[12])
                sfm_1.append(j[13])
                sp_rolloff_1.append(j[14])
                sp_bandwidth_1.append(j[15])
                label_1.append(label)
            elif label=='f':
                id_0.append(id_j)
                mean_0.append(j[1])
                sd_0.append(j[2])
                median_0.append(j[3])
                q25_0.append(j[4])
                q75_0.append(j[5])
                iqr_0.append(j[6])
                skew_0.append(j[7])
                kurt_0.append(j[8])
                mode_0.append(j[9])
                low_0.append(j[10])
                peak_0.append(j[11])
                sp_cent_0.append(j[12])
                sfm_0.append(j[13])
                sp_rolloff_0.append(j[14])
                sp_bandwidth_0.append(j[15])
                label_0.append(label)
csv_label_1=({"ID":id_1,'meanfreq':mean_1,'sd':sd_1,'median':median_1,'Q25':q25_1,'Q75':q75_1,'iqr':iqr_1,'skew':skew_1,'kurt':kurt_1,'mode':mode_1,'low':low_1,'peak':peak_1,'centroid':sp_cent_1,'flatness':sfm_1,'spectral rolloff':sp_rolloff_1,'spectral bandwidth':sp_bandwidth_1,"LABEL":label_1}) 
csv_label_0=({"ID":id_0,'meanfreq':mean_0,'sd':sd_0,'median':median_0,'Q25':q25_0,'Q75':q75_0,'iqr':iqr_0,'skew':skew_0,'kurt':kurt_0,'mode':mode_0,'low':low_0,'peak':peak_0,'centroid':sp_cent_0,'flatness':sfm_0,'spectral rolloff':sp_rolloff_0,'spectral bandwidth':sp_bandwidth_0,"LABEL":label_0})
csv_label_1=pd.DataFrame(csv_label_1)
csv_label_0=pd.DataFrame(csv_label_0)
df=pd.concat([csv_label_1,csv_label_0])
df.index=range(len(df))
df.head()

In [ ]:
df.to_csv("C:\\Users\\SAGAR\\Desktop\\Final Year Project\\csv_dataset\\dataset.csv",index=False)

# Thank you!!!!